In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import random_split, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [3]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

data_dir = "/content/drive/MyDrive/DataSet"  # Update path
full_dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

class_names = full_dataset.classes
print(f"Classes: {class_names}")
print(f"Train: {train_size} | Test: {test_size}")


Classes: ['Acne', 'Athlete-foot', 'Contact Dermatitis', 'Eczema', 'Folliculitis', 'Impetigo', 'Lupus', 'Milia', 'Psoriasis', 'Rosacea', 'Scabies Lyme Disease and other Infestations and Bites', 'Seborrh_Keratoses', 'Shingles', 'Tinea Ringworm Candidiasis', 'Vitiligo']
Train: 4808 | Test: 1202


In [4]:
model = models.vit_b_16(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Modify classifier
num_ftrs = model.heads.head.in_features
model.heads.head = nn.Linear(num_ftrs, len(class_names))

# Unfreeze classifier head
for param in model.heads.head.parameters():
    param.requires_grad = True

model = model.to(device)
print("Model ready.")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:01<00:00, 196MB/s]


Model ready.


In [ ]:
save_path = "/content/drive/MyDrive/vit_finetuned_final(100).pth"
patience = 5
best_loss = float('inf')
early_stop_counter = 0

def train_model(model, criterion, optimizer, num_epochs=100):
    global best_loss, early_stop_counter
    history = {'train_loss': [], 'test_loss': [], 'train_acc': [], 'test_acc': []}

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        for phase, loader in zip(['train', 'test'], [train_loader, test_loader]):
            model.train() if phase == 'train' else model.eval()
            running_loss, correct, total = 0.0, 0, 0

            for inputs, labels in loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                correct += torch.sum(preds == labels.data)
                total += labels.size(0)

            epoch_loss = running_loss / len(loader.dataset)
            epoch_acc = (correct.double() / len(loader.dataset)) * 100
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())

            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}%")

            if phase == 'test':
                if epoch_loss < best_loss:
                    best_loss = epoch_loss
                    torch.save(model.state_dict(), save_path)
                    print("Model saved!")
                    early_stop_counter = 0
                else:
                    early_stop_counter += 1
                    if early_stop_counter >= patience:
                        print("Early stopping.")
                        return history
    return history


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.heads.head.parameters(), lr=0.0001)


In [ ]:
history = train_model(model, criterion, optimizer)


Epoch 1/100
Train Loss: 2.3217 | Acc: 33.19%
Test Loss: 1.9368 | Acc: 54.66%
Model saved!
Epoch 2/100
Train Loss: 1.7125 | Acc: 59.46%
Test Loss: 1.5105 | Acc: 63.56%
Model saved!
Epoch 3/100
Train Loss: 1.3979 | Acc: 65.50%
Test Loss: 1.2809 | Acc: 67.30%
Model saved!
Epoch 4/100
Train Loss: 1.2149 | Acc: 68.57%
Test Loss: 1.1326 | Acc: 70.30%
Model saved!
Epoch 5/100
Train Loss: 1.0938 | Acc: 71.03%
Test Loss: 1.0340 | Acc: 73.04%
Model saved!
Epoch 6/100
Train Loss: 1.0058 | Acc: 73.25%
Test Loss: 0.9559 | Acc: 75.12%
Model saved!
Epoch 7/100
Train Loss: 0.9369 | Acc: 75.25%
Test Loss: 0.8981 | Acc: 75.79%
Model saved!
Epoch 8/100
Train Loss: 0.8803 | Acc: 76.29%
Test Loss: 0.8509 | Acc: 76.79%
Model saved!
Epoch 9/100
Train Loss: 0.8332 | Acc: 77.43%
Test Loss: 0.8109 | Acc: 77.95%
Model saved!
Epoch 10/100
Train Loss: 0.7942 | Acc: 78.64%
Test Loss: 0.7778 | Acc: 78.70%
Model saved!
Epoch 11/100
Train Loss: 0.7594 | Acc: 79.20%
Test Loss: 0.7492 | Acc: 79.53%
Model saved!
Epoch 12

In [5]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load model weights safely on CPU
model.load_state_dict(torch.load("/content/drive/MyDrive/vit_finetuned_final(100).pth", map_location=torch.device('cpu')))
model.to(device)
model.eval()


VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [6]:
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return 100 * correct / total


In [8]:
final_train_acc = evaluate_model(model, train_loader)
final_test_acc = evaluate_model(model, test_loader)

print(f"\nFinal Training Accuracy: {final_train_acc:.2f}%")
print(f"Final Testing Accuracy: {final_test_acc:.2f}%")



Final Training Accuracy: 93.57%
Final Testing Accuracy: 93.84%


In [9]:
final_model_path = "/content/drive/MyDrive/vit_finetuned_final(100).pth"
torch.save(model.state_dict(), final_model_path)
print(f" Final model manually saved to: {final_model_path}")


 Final model manually saved to: /content/drive/MyDrive/vit_finetuned_final(100).pth
